# Options Pricing

_UNDER CONSTRUCTION_


- Binomial pricing
- Simulation
- the Greeks


In [ ]:
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
from typing import List, Tuple, Any, Dict
from finds.database import SQL, RedisDB
from finds.structured import Stocks, Signals, SignalsFrame
from finds.busday import BusDay
from finds.misc import Show
from secret import credentials, paths, CRSP_DATE
show = Show(ndigits=4, latex=None)
pd.set_option('display.max_rows', None)
VERBOSE = 0
#%matplotlib qt
LAST_DATE = CRSP_DATE

In [ ]:
# open connections
imgdir = paths['images']
sql = SQL(**credentials['sql'], verbose=VERBOSE)
user = SQL(**credentials['user'], verbose=VERBOSE)
rdb = RedisDB(**credentials['redis'])
bd = BusDay(sql, verbose=VERBOSE)
signals = Signals(user, verbose=VERBOSE)
outdir = paths['images'] / 'glm'